In [2]:
from scipy.sparse import rand, csr_matrix
from sklearn.preprocessing import normalize, binarize
import numpy as np

# Implicit feedback Item-based CF

In [3]:
# матрица процентов видео просмотров
ui_m = rand(10000, 1000, 0.01, 'csr')

In [4]:
# нормализуем по пользователям
n_ui_m = normalize(ui_m)

In [6]:
# находим матрицу схожести item-item
from sklearn.metrics.pairwise import cosine_similarity
ii_sim_m = cosine_similarity(n_ui_m.T, dense_output=False).tocsr()

In [7]:
# вспомогательные функции, которые могут пригодиться при построении Item-based CF
def nullify_main_diagonal(m):
    positions = range(m.shape[0])
    eye = csr_matrix((np.ones(len(positions)), (positions, positions)), m.shape)
    return m - m.multiply(eye)


def get_topk(matrix, top, axis=1):
    """Converts source matrix to Top-K matrix
    where each row or column contains only top K values

    :param matrix: source matrix
    :param top: number of top items to be stored
    :param axis: 0 - top by column, 1 - top by row
    :return:
    """
    rows = []
    cols = []
    data = []

    if axis == 0:
        matrix = matrix.T.tocsr()

    for row_id, row in enumerate(matrix):
        if top is not None and row.nnz > top:
            top_args = np.argsort(row.data)[-top:]

            rows += [row_id] * top
            cols += row.indices[top_args].tolist()
            data += row.data[top_args].tolist()
        elif row.nnz > 0:
            rows += [row_id] * row.nnz
            cols += row.indices.tolist()
            data += row.data.tolist()

    topk_m = csr_matrix((data, (rows, cols)), (matrix.shape[0], matrix.shape[1]))

    if axis == 0:
        topk_m = topk_m.T.tocsr()

    return topk_m

In [8]:
# храним только top-30
ii_sim_m = nullify_main_diagonal(ii_sim_m)
ii_sim_m = get_topk(ii_sim_m, top=30)

In [11]:
# нормализуем схожести
ii_sim_m = normalize(ii_sim_m)

In [15]:
# находим рекомендации для пользователя uid
uid = 0
ui_m[uid]

<1x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 11 stored elements in Compressed Sparse Row format>

In [19]:
recs = n_ui_m[uid].dot(ii_sim_m.T)  # надо повернуть правильным краем
recs

<1x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 281 stored elements in Compressed Sparse Row format>

In [20]:
# напечатаем top-5 рекомедаций
TOP=5

for arg_id in np.argsort(recs.data)[-TOP:][::-1]:
    iid = recs.indices[arg_id]
    score = recs.data[arg_id]
    print(iid, score)

525 0.207314854948
359 0.204668280459
683 0.190652357752
809 0.17794113342
561 0.151127541368


In [23]:
# объясним рекомендацию пользователю
components = n_ui_m[uid].multiply(ii_sim_m[525])
max_comp_arg_id = np.argmax(components.data)
max_comp_id = components.indices[max_comp_arg_id]
print("Мы рекомендовали вам фильм 525, потому что вы смотрели фильм %s" % max_comp_id)

Мы рекомендовали вам фильм 525, потому что вы смотрели фильм 288


In [24]:
ui_m[uid, 288]

0.66164321686679062

In [25]:
ui_m[uid].data

array([ 0.5680924 ,  0.26633753,  0.29952498,  0.15172373,  0.14634499,
        0.66164322,  0.54244971,  0.36841495,  0.794185  ,  0.75723378,
        0.78673385])

In [28]:
ui_m[uid, components.indices].data

array([ 0.36841495,  0.54244971,  0.66164322,  0.14634499,  0.15172373])